In [1]:
# Enable autoreload
%load_ext autoreload
%autoreload 2

# Conversion notebook

To test our hypothesis in a completely black-box scenario, we have generated a CIFAR10 classifier using a MLaaS system. This MLaaS model expects its inputs to be .jpg files. So in order to test the generated adversarial examples we will need to convert them to images firts.

In [2]:
import os
os.chdir('../')

In [21]:
import json
import torch
import torchvision
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import transforms

from zest import utils
from zest import model

In [15]:
base_dir = 'data/advex100'
os.makedirs(base_dir, exist_ok=True)
base_save_pth = base_dir + '/victim_{}_advex_{}.jpg'
orig_save_pth = base_dir + '/original_{}.jpg'

In [5]:
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], 
    std=[0.229, 0.224, 0.225]
)
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255],
    std=[1/0.229, 1/0.224, 1/0.255]
)
transform = transforms.Compose([transforms.ToTensor(), normalize])

In [6]:
def show_torch_float(i):
    a = transforms.functional.to_pil_image(i.to(torch.uint8))
    plt.imshow(a)
    plt.show()

def show_torch_advex(img):
    # img = img / 2 + 0.5     # unnormalize
    img = inv_normalize(img)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

## Load data

In [7]:
dataset = 'CIFAR10'
dataset_class = eval(f"torchvision.datasets.{dataset}")
data = dataset_class(root='./data', train=False, download=True)

Files already downloaded and verified


## Adversarial examples

In [8]:
transfer_file = 'results/transfer_densenet121_densenet161_densenet169_googlenet_inception_v3_mobilenet_v2_resnet18_resnet34_resnet50_vgg11_bn_vgg13_bn_vgg16_bn_vgg19_bn.npy'
advex_file = 'results/advex_densenet121_densenet161_densenet169_googlenet_inception_v3_mobilenet_v2_resnet18_resnet34_resnet50_vgg11_bn_vgg13_bn_vgg16_bn_vgg19_bn.npy'
agree_file = 'results/agree_CIFAR10_testset_100_samples_densenet121_densenet161_densenet169_googlenet_inception_v3_mobilenet_v2_resnet18_resnet34_resnet50_vgg11_bn_vgg13_bn_vgg16_bn_vgg19_bn.npy'

# Load data points to attack
agree_data = np.load(agree_file, allow_pickle=True).item()
advex_res = np.load(advex_file, allow_pickle=True).item()
transfer_res = np.load(transfer_file, allow_pickle=True).item()

In [9]:
agree_idxs = agree_data['idxs']
agree_imgs = agree_data['imgs']
agree_labels = agree_data['labels']
print(agree_imgs.shape)

(100, 3, 32, 32)


We first save all the original images used to compute the adversarial examples as .jpg files.

In [16]:
unn_imgs = [data[i][0] for i in agree_idxs]
print(len(unn_imgs))
print(type(unn_imgs[0]))

# Save all original images
for i, img in enumerate(unn_imgs):
    img.save(orig_save_pth.format(i))

unnormalized_imgs = torch.stack([
    transforms.functional.pil_to_tensor(unnormalized_img) for unnormalized_img in unn_imgs
]).float()
print(unnormalized_imgs.shape)

100
<class 'PIL.Image.Image'>
torch.Size([100, 3, 32, 32])


### Check

In [11]:
base_model_path = '/home/giorgioseveri/projects/advml/lemon/cifar10_models/state_dicts/{}.pt'
# Utility to load a model from disk
def load_model(m):
    arch = eval(f"model.{m}")
    net = arch()
    net.load_state_dict(torch.load(base_model_path.format(m)))
    net = net.eval()
    return net

In [12]:
# Each entry in the dictionary is a numpy array of shape (num_advexes, 3, 32, 32)
for victim, advexes in advex_res.items():
    print(victim)
    print(np.min(advexes), np.max(advexes))
    print(np.min(agree_imgs), np.max(agree_imgs))

    v_net = load_model(victim)
    v_preds_adv = torch.argmax(v_net(torch.tensor(advexes)), dim=-1).numpy()
    v_preds = torch.argmax(v_net(torch.tensor(agree_imgs)), dim=-1).numpy()

    print('linf agree-advexes:', np.max(np.abs(agree_imgs - advexes)))
    print('linf agree-unnormalized', np.max(np.abs(agree_imgs - unnormalized_imgs.numpy())))

    print('check:', v_preds)
    print('adversarial:', v_preds_adv)

    v_preds_unn = torch.argmax(v_net(unnormalized_imgs), dim=-1).numpy()
    print('unnormalized: ', v_preds_unn)

    print('Check correct:', sum(v_preds == agree_labels))
    print('After evasion:', sum(v_preds_adv == agree_labels))
    print('Unnormalized correct:', sum(v_preds_unn == agree_labels))

    unn_advexes = torch.stack([inv_normalize(torch.tensor(a)) for a in advexes])
    unn_advexes_pil = [transforms.functional.to_pil_image(unn_adv) for unn_adv in unn_advexes]

    # Save the adversarial examples as jpg files
    for i, adv in enumerate(unn_advexes_pil):
        adv.save(base_save_pth.format(victim, i))

    unn_advexes_int = torch.stack([
        transforms.functional.pil_to_tensor(uap) for uap in unn_advexes_pil
    ])
    print('linf unn_advexes_int - unnormalized', np.max(np.abs(unn_advexes_int.numpy() - unnormalized_imgs.numpy())))
    assert unn_advexes_int.shape == (100, 3, 32, 32)

    unn_advexes_renorm = torch.stack([transform(u) for u in unn_advexes_pil])
    assert unn_advexes_renorm.shape == (100, 3, 32, 32)
    v_preds_renorm = torch.argmax(v_net(unn_advexes_renorm), dim=-1).numpy()
    print('After evasion renormalized:', sum(v_preds_renorm == agree_labels))
    print('Evasion renormalized:', v_preds_renorm)

    check_renorm = torch.stack([inv_normalize(torch.tensor(a)) for a in agree_imgs])
    assert check_renorm.shape == (100, 3, 32, 32)
    check_renorm_pil = [transforms.functional.to_pil_image(c) for c in check_renorm]
    check_renorm = torch.stack([ transform(c) for c in check_renorm_pil ])
    assert check_renorm.shape == (100, 3, 32, 32)
    v_preds_renorm_check = torch.argmax(v_net(check_renorm), dim=-1).numpy()
    print('Check renormalized:', sum(v_preds_renorm_check == agree_labels))
    print(v_preds_renorm_check)

    print('-'*80)
    print()

densenet121
-2.117904 2.64
-2.117904 2.64
linf agree-advexes: 0.100000024
linf agree-unnormalized 252.7511
check: [3 8 8 6 6 1 6 3 1 0 9 5 7 9 8 5 7 8 6 7 4 9 5 2 4 0 9 6 6 5 5 9 4 9 5 4 6
 5 6 0 9 3 7 6 9 8 8 8 7 7 6 6 2 1 2 3 7 6 8 8 0 2 9 3 8 8 1 1 7 2 5 8 9 0
 3 8 6 4 6 0 0 7 4 5 6 3 1 1 3 6 8 7 4 0 2 1 3 0 4 7]
adversarial: [5 1 1 4 5 9 2 5 9 3 1 3 5 1 0 7 1 9 8 4 2 1 4 5 3 6 3 3 5 9 2 0 5 5 9 1 7
 1 3 9 1 2 5 2 6 6 0 2 4 5 1 2 6 8 1 1 4 3 0 7 9 5 0 8 7 3 9 6 3 7 9 9 1 1
 6 9 9 2 2 4 2 3 7 9 5 6 9 9 6 4 1 5 7 2 3 9 9 8 0 1]
unnormalized:  [8 9 9 3 9 3 9 9 9 5 9 5 3 9 9 5 9 9 9 3 5 3 9 8 5 9 9 3 9 9 9 9 9 9 5 9 3
 5 9 9 9 9 8 8 9 9 9 9 3 8 3 9 3 9 9 9 9 9 9 9 9 8 9 5 9 9 3 9 5 9 9 9 9 9
 3 9 9 5 3 9 3 9 9 9 3 9 9 9 9 3 9 3 9 9 9 9 9 9 3 3]
Check correct: 100
After evasion: 0
Unnormalized correct: 14
linf unn_advexes_int - unnormalized 255.0
After evasion renormalized: 8
Evasion renormalized: [5 8 1 2 5 1 0 5 1 3 1 3 1 1 0 3 1 0 8 0 2 1 4 5 2 3 3 2 1 9 2 0 5 5 9 1 1
 1 3 1 3 2 5 2 3

#### Create jsonl file

We need to create a jsonl file containig the path to each jpg image we saved. This will be used by the Batch Prediction API.

In [25]:
jsonl_file = 'data/advex100.jsonl'
remote_base_pth = 'gs://bad-lemon-vcm/advex100/'
# Example
# {"content": "gs://sourcebucket/datasets/images/source_image.jpg", "mimeType": "image/jpeg"}

with open(jsonl_file, 'w') as mf:
    for i in range(len(unn_imgs)):
        cur_name = remote_base_pth + 'original_{}.jpg'.format(i)
        line_base = {"content": cur_name, "mimeType": "image/jpeg"}
        mf.write(json.dumps(line_base) + '\n')
        
    for victim, _ in advex_res.items():
        for i in range(len(unn_imgs)):
            cur_name = remote_base_pth + 'victim_{}_advex_{}.jpg'.format(victim, i)
            line_base = {"content": cur_name, "mimeType": "image/jpeg"}
            mf.write(json.dumps(line_base) + '\n')